In [170]:
import pandas as pd
import numpy as np

In [180]:
filenames= ["Mon","Tue","Wed","Thur","Fri"]

csvs={}
slotof={}
slotdayof={}
roomof={}
for filename in filenames:
    csv = pd.read_csv("csvs/"+filename+".csv")
    
    slots = list(csv.keys())[1:]
    for slot in slots:
        # For use in making entries -
        slotday=filename+"_"+slot
        for i,row in csv[['Room',slot]].dropna().iterrows():
            r = row[0]
            c = row[1]
            if( c in roomof ):
                roomof[c].add(r);
            else:
                roomof[c]=set([r])
            
            # Just for listing the multislots
            if( c in slotof ):
                slotof[c].add(slot);
            else:
                slotof[c]=set([slot])
            
            if( c in slotdayof ):
#                 if(slotday in slotdayof[c]):  
#                     print("Duplicate entry, Must have multirooms",slotday,c)
                slotdayof[c].add(slotday);
            else:
                slotdayof[c]=set([slotday])
    csvs[filename]=csv
# print("MultiSlot Courses-",[x for x in slotof.items() if(len(x[1]) != 1)])
print("SingleSlot Courses-",[x for x in slotof.items() if(len(x[1]) == 1)])
# print("MultiRoom Courses-",[x for x in roomof.items() if(len(x[1]) != 1)])

SingleSlot Courses- [('CE220', {'B'}), ('CE606', {'B'}), ('CE525', {'B'}), ('CE612', {'B'}), ('CE555', {'B'}), ('ME222', {'B'}), ('ME421', {'B'}), ('EE664', {'C'}), ('EE671', {'C'}), ('CE206', {'C'}), ('CE505', {'C'}), ('CE646', {'C'}), ('CE552', {'C'}), ('CE616', {'C'}), ('PH443', {'C'}), ('ME223', {'C'}), ('CE205', {'D'}), ('CE643', {'D'}), ('CE510', {'D'}), ('ME224', {'D'}), ('EE213M', {'F'}), ('EE322M', {'G'}), ('PH382M', {'G'}), ('ME321M', {'G'}), ('EE648', {'D1'}), ('EE351', {'D1'}), ('EE645', {'D1'}), ('CE595', {'D1'}), ('ME323', {'D1'}), ('EE668', {'C1'}), ('EE340', {'C1'}), ('EE634', {'C1'}), ('CE607', {'C1'}), ('CE664', {'C1'}), ('CE614', {'C1'}), ('ME324', {'C1'}), ('EE667', {'B1'}), ('EE360', {'B1'}), ('EE621', {'B1'}), ('EE653', {'B1'}), ('CE564', {'B1'}), ('CE628', {'B1'}), ('ME322', {'B1'}), ('ME321', {'A1'}), ('CE201', {'A'}), ('CE504', {'A'}), ('HS505', {'A'}), ('ME221', {'A'}), ('EE337', {'A1'}), ('EE385', {'A1'}), ('EE625', {'A1'}), ('CE514', {'A1'}), ('CE556', {'A1'

In [181]:
# Course	ClassSlot	Classroom	division	Day
final = []
days={"Mon":"Monday", "Tue":"Tuesday", "Wed":"Wednesday", "Thur":"Thursday", "Fri":"Friday" }
slotbased = pd.read_csv("csvs/Slotbased.csv")

# Normal, fixed slot courses-
for k,day in days.items():
    for i,csrd in slotbased.iterrows():
        final.append(list(csrd)+[day])
print("Normal ScheduledIn entries: ",len(final))

# Variable slot courses- (some are fixed though)
for csd in slotdayof.items():
    for s in csd[1]:
        for r in roomof[csd[0]]: #There are multiRoom courses as well.
            sd = s.split("_")
            final.append([csd[0],sd[1],r,'NA',days[sd[0]]])

print("Total ScheduledIn entries: ",len(final))
final[0:1630:200]

Normal ScheduledIn entries:  860
Total ScheduledIn entries:  1494


[['BT101', 'D', 'L2', 'I&III', 'Monday'],
 ['CH233', 'D', '4103', nan, 'Tuesday'],
 ['CL207', 'D', '4203', nan, 'Wednesday'],
 ['CS204', 'D', '1201', nan, 'Thursday'],
 ['HS237', 'E', '4005', nan, 'Friday'],
 ['PH304', 'C', '4003', 'NA', 'Tuesday'],
 ['CE607', 'C1', '3102', 'NA', 'Tuesday'],
 ['CE304', 'A1', '4211', 'NA', 'Tuesday']]

In [182]:
#Course entries - comes from slotdayof and slotbased (with division)
# course_code, division
courses = set()
division="NA"
for c in slotdayof.items():
    courses.add( (c[0],division) )
print(len(slotdayof))
print(len(courses))

for i,csrd in slotbased.iterrows():
    c = csrd[0]
    d = csrd[3]
    if( (c,d) in courses):
        print("Duplicate!",c,d)
    else:
        courses.add((c,d))

164
164


In [155]:
#Slot entries
final_slots=[]
times=[]
slots_all=pd.read_csv("csvs/slots.csv",header=None)
for i,r in slots_all.iterrows():
    lr= list(r)
    if(i<1):
        times=lr[1:]
    else:
        day = lr[0]
        for t,l in enumerate(lr[1:]):
            if(type(l)==str): #ignore nan
                final_slots.append([l,day]+times[t].split("-"))